In [122]:
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, multilabel_confusion_matrix
from sklearn.metrics.pairwise import cosine_distances
from sklearn import model_selection
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dasamso3/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

class FasterMorphology:
    
    def __init__(self):
        self.__morph = MorphAnalyzer()
        self.__cache = {}
    
    def parse(self, word: str):
        if word in self.__cache:
            return self.__cache[word]
        else:
            analysis = self.__morph.parse(word)
            self.__cache[word] = analysis[0]
            return analysis[0]


morph = FasterMorphology()

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word).normal_form for word in words if word]

    return words

In [2]:
import requests
from lxml import etree, html
from time import sleep

In [29]:
links = []

In [30]:
for topic in [33, 37, 10, 13, 36, 12, 34, 39]:
    for letter in 'абвгдезиклмнпростуфхцшщчуэюя':
        for page_num in range(0, 30, 1):
            r = requests.get(f'https://www.krugosvet.ru/taxonomy/term/{topic}/{letter}?page={page_num}')
            if r.status_code != 200:
                continue
            page = html.fromstring(r.text)
            hrefs = page.xpath("//div[@class='article-teaser']/a/@href")
            links += ['https://www.krugosvet.ru'+href for href in hrefs]
    print('after topic - ', topic, ', links collected - ', len(links))

after topic -  33 , links collected -  586
after topic -  37 , links collected -  1523
after topic -  10 , links collected -  1683
after topic -  13 , links collected -  3444
after topic -  36 , links collected -  3803
after topic -  12 , links collected -  3875
after topic -  34 , links collected -  4103
after topic -  39 , links collected -  6083


In [38]:
texts = []

for link in tqdm(links):
    r = requests.get(link)
    if r.status_code != 200:
        continue
    page = html.fromstring(r.text)
    text = "".join(page.xpath("//div[@id='article-content']/div[@class='body']//text()"))
    texts.append(text)




  0%|          | 0/6083 [00:00<?, ?it/s]


  0%|          | 1/6083 [00:00<36:10,  2.80it/s]


  0%|          | 2/6083 [00:00<35:56,  2.82it/s]


  0%|          | 3/6083 [00:01<36:05,  2.81it/s]


  0%|          | 4/6083 [00:01<38:01,  2.66it/s]


  0%|          | 5/6083 [00:01<38:25,  2.64it/s]


  0%|          | 6/6083 [00:02<38:26,  2.63it/s]


  0%|          | 7/6083 [00:02<39:06,  2.59it/s]


  0%|          | 8/6083 [00:03<39:11,  2.58it/s]


  0%|          | 9/6083 [00:03<36:59,  2.74it/s]


  0%|          | 10/6083 [00:03<36:39,  2.76it/s]


  0%|          | 11/6083 [00:04<36:33,  2.77it/s]


  0%|          | 12/6083 [00:04<34:18,  2.95it/s]


  0%|          | 13/6083 [00:04<34:58,  2.89it/s]


  0%|          | 14/6083 [00:05<35:19,  2.86it/s]


  0%|          | 15/6083 [00:05<38:01,  2.66it/s]


  0%|          | 16/6083 [00:06<43:16,  2.34it/s]


  0%|          | 17/6083 [00:06<44:03,  2.29it/s]


  0%|          | 18/6083 [00:06<39:41,  2.55it/s]


  0%|          | 19/6083 [0

  2%|▏         | 134/6083 [00:57<41:02,  2.42it/s]


  2%|▏         | 135/6083 [00:57<41:31,  2.39it/s]


  2%|▏         | 136/6083 [00:57<39:02,  2.54it/s]


  2%|▏         | 137/6083 [00:58<38:50,  2.55it/s]


  2%|▏         | 138/6083 [00:58<39:38,  2.50it/s]


  2%|▏         | 139/6083 [00:59<43:21,  2.28it/s]


  2%|▏         | 140/6083 [00:59<44:08,  2.24it/s]


  2%|▏         | 141/6083 [01:00<45:02,  2.20it/s]


  2%|▏         | 142/6083 [01:00<45:26,  2.18it/s]


  2%|▏         | 143/6083 [01:01<46:24,  2.13it/s]


  2%|▏         | 144/6083 [01:01<43:00,  2.30it/s]


  2%|▏         | 145/6083 [01:01<41:23,  2.39it/s]


  2%|▏         | 146/6083 [01:03<1:06:43,  1.48it/s]


  2%|▏         | 147/6083 [01:03<59:06,  1.67it/s]  


  2%|▏         | 148/6083 [01:03<52:58,  1.87it/s]


  2%|▏         | 149/6083 [01:04<55:37,  1.78it/s]


  2%|▏         | 150/6083 [01:04<52:14,  1.89it/s]


  2%|▏         | 151/6083 [01:05<57:27,  1.72it/s]


  2%|▏         | 152/6083 [01:06<53:57,  1

  7%|▋         | 397/6083 [03:01<41:46,  2.27it/s]


  7%|▋         | 398/6083 [03:02<45:09,  2.10it/s]


  7%|▋         | 399/6083 [03:02<44:20,  2.14it/s]


  7%|▋         | 400/6083 [03:02<42:35,  2.22it/s]


  7%|▋         | 401/6083 [03:03<43:21,  2.18it/s]


  7%|▋         | 402/6083 [03:03<41:42,  2.27it/s]


  7%|▋         | 403/6083 [03:04<39:57,  2.37it/s]


  7%|▋         | 404/6083 [03:04<38:34,  2.45it/s]


  7%|▋         | 405/6083 [03:04<37:23,  2.53it/s]


  7%|▋         | 406/6083 [03:05<38:19,  2.47it/s]


  7%|▋         | 407/6083 [03:05<43:00,  2.20it/s]


  7%|▋         | 408/6083 [03:06<40:13,  2.35it/s]


  7%|▋         | 409/6083 [03:06<38:42,  2.44it/s]


  7%|▋         | 410/6083 [03:07<39:13,  2.41it/s]


  7%|▋         | 411/6083 [03:07<37:23,  2.53it/s]


  7%|▋         | 412/6083 [03:07<36:37,  2.58it/s]


  7%|▋         | 413/6083 [03:08<34:17,  2.76it/s]


  7%|▋         | 414/6083 [03:08<34:05,  2.77it/s]


  7%|▋         | 415/6083 [03:08<34:01,  2.78i

 11%|█         | 661/6083 [04:55<38:31,  2.35it/s]


 11%|█         | 662/6083 [04:55<38:01,  2.38it/s]


 11%|█         | 663/6083 [04:56<40:21,  2.24it/s]


 11%|█         | 664/6083 [04:57<51:05,  1.77it/s]


 11%|█         | 665/6083 [04:57<45:29,  1.98it/s]


 11%|█         | 666/6083 [04:57<43:38,  2.07it/s]


 11%|█         | 667/6083 [04:58<40:40,  2.22it/s]


 11%|█         | 668/6083 [04:58<42:26,  2.13it/s]


 11%|█         | 669/6083 [04:59<40:02,  2.25it/s]


 11%|█         | 670/6083 [04:59<38:36,  2.34it/s]


 11%|█         | 671/6083 [04:59<37:21,  2.41it/s]


 11%|█         | 672/6083 [05:00<35:54,  2.51it/s]


 11%|█         | 673/6083 [05:00<37:40,  2.39it/s]


 11%|█         | 674/6083 [05:01<37:22,  2.41it/s]


 11%|█         | 675/6083 [05:01<35:22,  2.55it/s]


 11%|█         | 676/6083 [05:01<37:03,  2.43it/s]


 11%|█         | 677/6083 [05:02<36:40,  2.46it/s]


 11%|█         | 678/6083 [05:02<36:06,  2.49it/s]


 11%|█         | 679/6083 [05:03<34:42,  2.59i

 15%|█▌        | 925/6083 [06:47<39:09,  2.20it/s]


 15%|█▌        | 926/6083 [06:47<40:19,  2.13it/s]


 15%|█▌        | 927/6083 [06:48<39:54,  2.15it/s]


 15%|█▌        | 928/6083 [06:48<38:42,  2.22it/s]


 15%|█▌        | 929/6083 [06:49<35:36,  2.41it/s]


 15%|█▌        | 930/6083 [06:49<36:09,  2.38it/s]


 15%|█▌        | 931/6083 [06:49<35:51,  2.39it/s]


 15%|█▌        | 932/6083 [06:50<33:48,  2.54it/s]


 15%|█▌        | 933/6083 [06:50<33:55,  2.53it/s]


 15%|█▌        | 934/6083 [06:51<36:20,  2.36it/s]


 15%|█▌        | 935/6083 [06:51<38:38,  2.22it/s]


 15%|█▌        | 936/6083 [06:52<37:38,  2.28it/s]


 15%|█▌        | 937/6083 [06:52<36:45,  2.33it/s]


 15%|█▌        | 938/6083 [06:52<36:32,  2.35it/s]


 15%|█▌        | 939/6083 [06:53<35:53,  2.39it/s]


 15%|█▌        | 940/6083 [06:53<35:21,  2.42it/s]


 15%|█▌        | 941/6083 [06:54<35:36,  2.41it/s]


 15%|█▌        | 942/6083 [06:54<34:22,  2.49it/s]


 16%|█▌        | 943/6083 [06:54<35:39,  2.40i

...

 79%|███████▉  | 4826/6083 [34:33<10:48,  1.94it/s]


 79%|███████▉  | 4827/6083 [34:34<10:08,  2.07it/s]


 79%|███████▉  | 4828/6083 [34:34<09:14,  2.26it/s]


 79%|███████▉  | 4829/6083 [34:34<08:18,  2.52it/s]


 79%|███████▉  | 4830/6083 [34:35<08:42,  2.40it/s]


 79%|███████▉  | 4831/6083 [34:35<08:15,  2.53it/s]


 79%|███████▉  | 4832/6083 [34:36<08:44,  2.38it/s]


 79%|███████▉  | 4833/6083 [34:36<08:13,  2.53it/s]


 79%|███████▉  | 4834/6083 [34:36<08:18,  2.50it/s]


 79%|███████▉  | 4835/6083 [34:37<09:04,  2.29it/s]


 80%|███████▉  | 4836/6083 [34:37<09:16,  2.24it/s]


 80%|███████▉  | 4837/6083 [34:38<09:05,  2.28it/s]


 80%|███████▉  | 4838/6083 [34:38<08:41,  2.39it/s]


 80%|███████▉  | 4839/6083 [34:39<10:04,  2.06it/s]


 80%|███████▉  | 4840/6083 [34:39<09:38,  2.15it/s]


 80%|███████▉  | 4841/6083 [34:40<08:50,  2.34it/s]


 80%|███████▉  | 4842/6083 [34:40<09:23,  2.20it/s]


 80%|███████▉  | 4843/6083 [34:41<08:43,  2.37it/s]


 80%|███████▉  | 4844/6083 [

 84%|████████▎ | 5086/6083 [36:29<06:39,  2.49it/s]


 84%|████████▎ | 5087/6083 [36:30<07:02,  2.36it/s]


 84%|████████▎ | 5088/6083 [36:30<07:24,  2.24it/s]


 84%|████████▎ | 5089/6083 [36:31<07:41,  2.16it/s]


 84%|████████▎ | 5090/6083 [36:31<07:55,  2.09it/s]


 84%|████████▎ | 5091/6083 [36:32<07:17,  2.27it/s]


 84%|████████▎ | 5092/6083 [36:32<07:22,  2.24it/s]


 84%|████████▎ | 5093/6083 [36:32<06:58,  2.37it/s]


 84%|████████▎ | 5094/6083 [36:33<06:57,  2.37it/s]


 84%|████████▍ | 5095/6083 [36:33<06:16,  2.63it/s]


 84%|████████▍ | 5096/6083 [36:34<06:14,  2.63it/s]


 84%|████████▍ | 5097/6083 [36:34<06:06,  2.69it/s]


 84%|████████▍ | 5098/6083 [36:34<06:43,  2.44it/s]


 84%|████████▍ | 5099/6083 [36:35<06:14,  2.63it/s]


 84%|████████▍ | 5100/6083 [36:35<06:33,  2.49it/s]


 84%|████████▍ | 5101/6083 [36:36<07:57,  2.06it/s]


 84%|████████▍ | 5102/6083 [36:36<08:35,  1.90it/s]


 84%|████████▍ | 5103/6083 [36:37<07:51,  2.08it/s]


 84%|████████▍ | 5104/6083 [

 88%|████████▊ | 5346/6083 [38:20<04:53,  2.51it/s]


 88%|████████▊ | 5347/6083 [38:20<04:45,  2.58it/s]


 88%|████████▊ | 5348/6083 [38:20<05:11,  2.36it/s]


 88%|████████▊ | 5349/6083 [38:21<04:38,  2.63it/s]


 88%|████████▊ | 5350/6083 [38:21<04:38,  2.63it/s]


 88%|████████▊ | 5351/6083 [38:22<04:47,  2.55it/s]


 88%|████████▊ | 5352/6083 [38:22<04:37,  2.64it/s]


 88%|████████▊ | 5353/6083 [38:22<04:29,  2.70it/s]


 88%|████████▊ | 5354/6083 [38:23<04:50,  2.51it/s]


 88%|████████▊ | 5355/6083 [38:23<04:38,  2.61it/s]


 88%|████████▊ | 5356/6083 [38:23<04:38,  2.61it/s]


 88%|████████▊ | 5357/6083 [38:24<04:53,  2.47it/s]


 88%|████████▊ | 5358/6083 [38:24<04:45,  2.54it/s]


 88%|████████▊ | 5359/6083 [38:25<04:43,  2.56it/s]


 88%|████████▊ | 5360/6083 [38:25<04:30,  2.68it/s]


 88%|████████▊ | 5361/6083 [38:25<04:46,  2.52it/s]


 88%|████████▊ | 5362/6083 [38:26<04:35,  2.62it/s]


 88%|████████▊ | 5363/6083 [38:26<05:13,  2.30it/s]


 88%|████████▊ | 5364/6083 [

 92%|█████████▏| 5606/6083 [40:10<03:15,  2.43it/s]


 92%|█████████▏| 5607/6083 [40:11<03:28,  2.29it/s]


 92%|█████████▏| 5608/6083 [40:11<03:20,  2.37it/s]


 92%|█████████▏| 5609/6083 [40:12<03:19,  2.38it/s]


 92%|█████████▏| 5610/6083 [40:12<03:06,  2.53it/s]


 92%|█████████▏| 5611/6083 [40:12<02:59,  2.62it/s]


 92%|█████████▏| 5612/6083 [40:13<02:53,  2.72it/s]


 92%|█████████▏| 5613/6083 [40:13<02:43,  2.88it/s]


 92%|█████████▏| 5614/6083 [40:13<02:44,  2.85it/s]


 92%|█████████▏| 5615/6083 [40:14<03:09,  2.48it/s]


 92%|█████████▏| 5616/6083 [40:14<03:18,  2.35it/s]


 92%|█████████▏| 5617/6083 [40:15<03:09,  2.46it/s]


 92%|█████████▏| 5618/6083 [40:15<02:54,  2.67it/s]


 92%|█████████▏| 5619/6083 [40:15<02:52,  2.69it/s]


 92%|█████████▏| 5620/6083 [40:16<02:53,  2.67it/s]


 92%|█████████▏| 5621/6083 [40:16<02:47,  2.75it/s]


 92%|█████████▏| 5622/6083 [40:16<03:01,  2.55it/s]


 92%|█████████▏| 5623/6083 [40:17<02:57,  2.59it/s]


 92%|█████████▏| 5624/6083 [

 96%|█████████▋| 5866/6083 [41:58<01:29,  2.44it/s]


 96%|█████████▋| 5867/6083 [41:58<01:25,  2.53it/s]


 96%|█████████▋| 5868/6083 [41:59<01:28,  2.44it/s]


 96%|█████████▋| 5869/6083 [41:59<01:25,  2.51it/s]


 96%|█████████▋| 5870/6083 [41:59<01:21,  2.62it/s]


 97%|█████████▋| 5871/6083 [42:00<01:23,  2.55it/s]


 97%|█████████▋| 5872/6083 [42:00<01:22,  2.57it/s]


 97%|█████████▋| 5873/6083 [42:01<01:28,  2.37it/s]


 97%|█████████▋| 5874/6083 [42:01<01:27,  2.39it/s]


 97%|█████████▋| 5875/6083 [42:01<01:20,  2.60it/s]


 97%|█████████▋| 5876/6083 [42:02<02:09,  1.60it/s]


 97%|█████████▋| 5877/6083 [42:03<01:50,  1.87it/s]


 97%|█████████▋| 5878/6083 [42:03<01:37,  2.11it/s]


 97%|█████████▋| 5879/6083 [42:03<01:29,  2.28it/s]


 97%|█████████▋| 5880/6083 [42:04<01:27,  2.33it/s]


 97%|█████████▋| 5881/6083 [42:04<01:22,  2.46it/s]


 97%|█████████▋| 5882/6083 [42:05<01:24,  2.37it/s]


 97%|█████████▋| 5883/6083 [42:05<01:20,  2.47it/s]


 97%|█████████▋| 5884/6083 [

In [ ]:
corpus_hum = open('corpus_hum.txt', 'w')

for text in texts:
    if len(text)>10:
        corpus_hum.write(text + '\n')

corpus_hum.close()

In [12]:
with open('data/corpus_hum.txt') as file:
    data = file.read().splitlines()

data_norm = [normalize(text) for text in data]

In [3]:
data_norm = [text for text in data_norm if text]

In [13]:
data_ling = data_norm

In [14]:
len(data_ling)

139935

### Getting the paraphrases


In [18]:
with open('data/paraphrases.xml', 'rb') as file:
    corpus_xml = html.fromstring(file.read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [19]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [20]:
data.head(5)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,"[полицейский, разрешить, стрелять, поражение, ...","[полиция, мочь, разрешить, стрелять, хулиган, ..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,"[право, полицейский, проникновение, жилища, ре...","[правило, внесудебный, проникновение, полицейс..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,"[президент, египет, ввести, чрезвычайный, поло...","[власть, египет, угрожать, ввести, страна, чре..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,"[вернуться, сирия, россиянин, волновать, вопро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,"[москва, сирия, вернуться, 2, самолёт, мчс, ро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."


### Vectorizing with pretrained w2v

In [21]:
model = gensim.models.KeyedVectors.load_word2vec_format('models/185/model.bin', binary=True)

In [22]:
mapping = {}

with open('ru-rnc.map.txt') as file:
    for line in file:
        ms, ud = line.strip('\n').split()
        mapping[ms] = ud

In [23]:
mapping

{'A': 'ADJ',
 'ADV': 'ADV',
 'ADVPRO': 'ADV',
 'ANUM': 'ADJ',
 'APRO': 'DET',
 'COM': 'ADJ',
 'CONJ': 'SCONJ',
 'INTJ': 'INTJ',
 'NONLEX': 'X',
 'NUM': 'NUM',
 'PART': 'PART',
 'PR': 'ADP',
 'S': 'NOUN',
 'SPRO': 'PRON',
 'UNKN': 'X',
 'V': 'VERB'}

In [25]:
!pip install pymystem3

In [26]:
from pymystem3 import Mystem
m = Mystem()

In [61]:
from typing import Tuple

from pymystem3 import Mystem

mystem = Mystem()


def analyze(text: str) -> Tuple[str, str]:
    for word_analysis in mystem.analyze(text):
        analysis = word_analysis.get('analysis')
        if analysis:
            if len(analysis) != 0:
                grammar = analysis[0].get('gr')
                lemma = analysis[0].get('lex')
                if grammar:
                    pos = grammar.split('=')[0].split(',')[0]
                    yield lemma, pos
                    continue
            yield word_analysis.get('word'), 'UNKN'

In [62]:
def normalize_mystem(text):
    tokens = [f'{word}_{pos}' for word, pos in analyze(text)]
    return tokens

In [63]:
normalize_mystem('Обучить классификатор парафразов на предобученной модели вам нужно будет дома')

['обучать_V',
 'классификатор_S',
 'парафраз_S',
 'на_PR',
 'предобученный_A',
 'модель_S',
 'вы_SPRO',
 'нужно_ADV',
 'быть_V',
 'дома_ADV']

In [65]:
data['text_1_norm']

0       [полицейский, разрешить, стрелять, поражение, ...
1       [право, полицейский, проникновение, жилища, ре...
2       [президент, египет, ввести, чрезвычайный, поло...
3       [вернуться, сирия, россиянин, волновать, вопро...
4       [москва, сирия, вернуться, 2, самолёт, мчс, ро...
                              ...                        
7222       [путин, освободить, должность, ряд, генералов]
7223    [облако, москва, день, победа, разогнать, девя...
7224    [люблян, отпраздновать, день, победа, вместе, ...
7225    [девять, самолёт, ввс, разогнать, облако, моск...
7226    [9, май, метрополитен, петербург, работать, кр...
Name: text_1_norm, Length: 7227, dtype: object

In [68]:
def get_embedding(text, model, dim):
    split_text = text.split()
    words = Counter(split_text)
    total = len(split_text)
    vectors = np.zeros((len(words), dim))

    
    for i ,word in enumerate(words):
        try:
            v = model[normalize_mystem(word)]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
            
    if vectors.any():
        vectors = np.average(vectors, axis=0)
    else:
        vectors = np.zeros((dim))
    
    return vectors

In [70]:
dim = 300
X_text_1_w2v_pretrained = np.zeros((len(data['text_1']), dim))
X_text_2_w2v_pretrained = np.zeros((len(data['text_2']), dim))

for i, text in enumerate(data['text_1'].values):
    X_text_1_w2v_pretrained[i] = get_embedding(text, model, dim)
    
for i, text in enumerate(data['text_2'].values):
    X_text_2_w2v_pretrained[i] = get_embedding(text, model, dim)

In [71]:
X_text_w2v_pretrained = np.concatenate([X_text_1_w2v_pretrained, X_text_2_w2v_pretrained], axis=1)

In [72]:
X_text_w2v_pretrained

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.00130854, -0.00500983, -0.00312244, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00192228, -0.0007066 ,  0.0022999 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00121496,  0.00508754, -0.00237417, ...,  0.        ,
         0.        ,  0.        ]])

In [73]:
y = data["label"].apply(lambda x: int(x)).to_numpy()

In [74]:

train_X, test_X, train_y, test_y = train_test_split(X_text_w2v_pretrained, y,random_state=1)

In [75]:
data["label"].isnull().values.any()

False

In [79]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)

In [187]:
def cv_train_model(X, y, X_test, y_test, folds=folds, plot_feature_importance=False, model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(y_test))
    
    valid_scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y), start=1):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        

        model.fit(X_train, y_train)      
        y_pred_test = model.predict(X_test)
        y_pred_valid = model.predict(X_valid)
        valid_score = accuracy_score(y_valid, y_pred_valid)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        valid_scores.append(valid_score)
        print(y_pred_test)
        prediction += y_pred_test
        
        
    prediction /= fold_n
    print('Cross Validation mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(valid_scores), np.std(valid_scores)))
    print(multilabel_confusion_matrix(y_test, y_pred_test))
    print(classification_report(y_test, y_pred_test, digits=3))
    

    return oof, prediction, valid_scores

In [88]:
model = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                              class_weight='balanced')
oof_lr, prediction_lr, scores = cv_train_model(train_X, train_y, test_X, test_y,  model=model,  folds=folds)

Cross Validation mean score: 0.3183, std: 0.0223.
[[[1068  110]
  [ 500  129]]

 [[ 982   88]
  [ 671   66]]

 [[ 334 1032]
  [  59  382]]]
              precision    recall  f1-score   support

          -1      0.540     0.205     0.297       629
           0      0.429     0.090     0.148       737
           1      0.270     0.866     0.412       441

    accuracy                          0.319      1807
   macro avg      0.413     0.387     0.286      1807
weighted avg      0.429     0.319     0.264      1807



In [193]:
cosine_distance_pretrW2V = cosine_distances(X_text_1_w2v_pretrained, X_text_2_w2v_pretrained)

### Vectorizing with Word2Vec

In [98]:
%time
my_w2v = gensim.models.Word2Vec(data_ling, size=100, sg=1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [128]:
my_w2v['василий']

/Users/grigoriyarshinov/anaconda3/envs/lab_8/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.21888523, -0.25314295, -0.54204416, -0.7659865 ,  0.34566462,
       -0.14228666, -0.3622128 , -0.18285461,  0.0182605 , -0.02044144,
        0.23904273, -0.31888098,  0.42699233,  0.24010819, -0.03073579,
        0.13736191, -0.16574311, -0.13785508,  0.01881371,  0.39281046,
       -0.15979706,  0.10957947, -0.141065  ,  0.06916313,  0.87987417,
        0.20997225,  0.35270667, -0.08845262,  0.68942237, -0.19881934,
        0.3988385 , -0.42675215, -0.30455214, -0.02313071, -0.05768723,
       -0.4996426 , -0.02740662,  0.6348216 ,  0.785284  , -0.3253529 ,
        0.23860641, -0.24263844, -0.29095143,  0.21784586,  0.44613126,
       -0.30447048,  0.12195291, -0.34106335, -0.51997733,  0.19045976,
       -0.5944905 , -0.2752096 ,  0.85414207, -0.6123254 , -0.12299555,
       -0.23889872, -0.22006494,  0.7256147 , -0.18735202, -0.20246156,
       -0.8191216 ,  0.69318193, -0.26231286,  0.14598867, -0.16590486,
       -0.7252886 ,  0.04954244, -0.28025138, -0.17698714,  0.28

In [100]:
dim = 100
X_text_1_w2v_my_model = np.zeros((len(data['text_1']), dim))
X_text_2_w2v_my_model = np.zeros((len(data['text_2']), dim))

for i, text in enumerate(data['text_1'].values):
    X_text_1_w2v_my_model[i] = get_embedding(text, my_w2v, dim)
    
for i, text in enumerate(data['text_2'].values):
    X_text_2_w2v_my_model[i] = get_embedding(text,my_w2v, dim)

/Users/grigoriyarshinov/anaconda3/envs/lab_8/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [101]:
X_text_w2v_my_model = np.concatenate([X_text_1_w2v_my_model, X_text_2_w2v_my_model], axis=1)

In [102]:
y = data["label"].apply(lambda x: int(x)).to_numpy()
train_X, test_X, train_y, test_y = train_test_split(X_text_w2v_my_model, y,random_state=1)


In [103]:
X_text_w2v_my_model

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [104]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)


In [105]:
model = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                              class_weight='balanced')
oof_lr, prediction_lr, scores = cv_train_model(train_X, train_y, test_X, test_y,  model=model,  folds=folds)

Cross Validation mean score: 0.3323, std: 0.0769.
[[[   0 1178]
  [   0  629]]

 [[1070    0]
  [ 737    0]]

 [[1366    0]
  [ 441    0]]]
              precision    recall  f1-score   support

          -1      0.348     1.000     0.516       629
           0      0.000     0.000     0.000       737
           1      0.000     0.000     0.000       441

    accuracy                          0.348      1807
   macro avg      0.116     0.333     0.172      1807
weighted avg      0.121     0.348     0.180      1807



/Users/grigoriyarshinov/anaconda3/envs/lab_8/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [200]:
cosine_distance_WV_MY = cosine_distances(X_text_1_w2v_my_model, X_text_2_w2v_my_model)

### Vectorizing FastText

In [108]:
fast_text = gensim.models.FastText([text for text in data_norm], size=50, 
                                   min_n=4, max_n=8) 
dim = 100
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1']), dim))
X_text_2_ft = np.zeros((len(data['text_2']), dim))

for i, text in enumerate(data['text_1'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)

for i, text in enumerate(data['text_2'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

/Users/grigoriyarshinov/anaconda3/envs/lab_8/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [109]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [110]:
cosine_distance_FT = cosine_distances(X_text_1_ft, X_text_2_ft)

In [111]:
cosine_distance_FT

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

### Vectorizing SVD

In [113]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'].apply(lambda x: ' '.join(x)), data['text_2_norm'].apply(lambda x: ' '.join(x))]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=1000,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [115]:
svd = TruncatedSVD(200)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm'].apply(lambda x: ' '.join(x))))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm'].apply(lambda x: ' '.join(x))))

X_text_svd = np.concatenate([X_text_1_svd, X_text_2_svd], axis=1)

In [116]:
X_text_svd.shape

(7227, 400)

In [117]:
y = data['label'].values
print(y.shape)

(7227,)


In [118]:
cosine_distance_SVD = cosine_distances(X_text_1_svd, X_text_2_svd)

### Vectorizing  NMF

In [119]:
model = NMF(n_components=2, init='random', random_state=0)
X_text_1_nmf = model.fit_transform(tfidf.transform(data['text_1_norm'].apply(lambda x: ' '.join(x)))) 
X_text_2_nmf = model.fit_transform(tfidf.transform(data['text_2_norm'].apply(lambda x: ' '.join(x))))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [120]:
X_text_nmf

array([[0.00239297, 0.02133561, 0.00280243, 0.00157717],
       [0.        , 0.01711455, 0.00196638, 0.00087914],
       [0.        , 0.02514677, 0.00263052, 0.00128497],
       ...,
       [0.        , 0.10220821, 0.00811775, 0.00390681],
       [0.0025552 , 0.08821695, 0.00811775, 0.00390681],
       [0.00118718, 0.03441192, 0.0010393 , 0.0009745 ]])

In [132]:
cosine_distances(X_text_1_nmf, X_text_2_nmf)

array([[0.41547065, 0.49263985, 0.46366735, ..., 0.46860971, 0.46860971,
        0.2389542 ],
       [0.50954816, 0.59184979, 0.56108193, ..., 0.56634129, 0.56634129,
        0.31599996],
       [0.50954816, 0.59184979, 0.56108193, ..., 0.56634129, 0.56634129,
        0.31599996],
       ...,
       [0.50954816, 0.59184979, 0.56108193, ..., 0.56634129, 0.56634129,
        0.31599996],
       [0.48452235, 0.56558949, 0.53525109, ..., 0.54043442, 0.54043442,
        0.29516619],
       [0.47979276, 0.56061644, 0.53036294, ..., 0.53553125, 0.53553125,
        0.29125512]])

In [133]:
cosine_distance_NMF = cosine_distances(X_text_1_nmf, X_text_2_nmf)

Classification

In [159]:
def get_diagonal(matrix: np.ndarray):
    needed_values = []
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[0]):
            if i == j:
                needed_values.append(matrix[i, j])
    return np.array(needed_values)

In [163]:
cosine_distance_NMF_ = get_diagonal(cosine_distance_NMF)

In [164]:
cosine_distance_FT_ = get_diagonal(cosine_distance_FT)

In [165]:
cosine_distance_SVD_ = get_diagonal(cosine_distance_SVD)

In [194]:
cosine_distance_pretrW2V_ = get_diagonal(cosine_distance_pretrW2V)

In [202]:
cosine_distance_WV_MY_ = get_diagonal(cosine_distance_WV_MY)

In [179]:
cosine_distance_FT_.reshape((-1, 1))

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [203]:
X = np.concatenate([cosine_distance_FT_.reshape((-1, 1)),
                    cosine_distance_NMF_.reshape((-1, 1)),
                    cosine_distance_SVD_.reshape((-1, 1)),
                    cosine_distance_pretrW2V_.reshape((-1, 1)),
                    cosine_distance_WV_MY_.reshape((-1, 1))],
                   axis=1)

In [204]:
X

array([[1.        , 0.41547065, 1.07252414, 1.        , 1.        ],
       [1.        , 0.59184979, 0.85830302, 1.        , 1.        ],
       [1.        , 0.56108193, 0.99567901, 1.        , 1.        ],
       ...,
       [1.        , 0.56634129, 0.91761793, 1.        , 1.        ],
       [1.        , 0.54043442, 0.88227229, 1.        , 1.        ],
       [1.        , 0.29125512, 1.05373866, 1.        , 1.        ]])

In [197]:
y.astype('int')

array([ 0,  0,  0, ..., -1, -1, -1])

In [206]:
train_X, test_X, train_y, test_y = train_test_split(X, y.astype('int'),random_state=1)

In [207]:
model = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                              class_weight='balanced')
oof_lr, prediction_lr, scores = cv_train_model(train_X, train_y, test_X, test_y,  model=model,  folds=folds)

[ 1 -1 -1 ...  1  0  1]
[ 1  0 -1 ...  0  0  0]
[ 1 -1 -1 ...  1  0  1]
[ 1 -1 -1 ...  1  0  1]
[ 1  0 -1 ...  1  0  0]
[ 1 -1 -1 ...  1  0  0]
[ 1  1 -1 ...  1  0  1]
[ 1 -1 -1 ...  1  0  1]
[ 1  0 -1 ...  1  0  1]
[ 1  0 -1 ...  1  0  0]
Cross Validation mean score: 0.3727, std: 0.0305.
[[[909 269]
  [370 259]]

 [[708 362]
  [426 311]]

 [[938 428]
  [263 178]]]
              precision    recall  f1-score   support

          -1      0.491     0.412     0.448       629
           0      0.462     0.422     0.441       737
           1      0.294     0.404     0.340       441

    accuracy                          0.414      1807
   macro avg      0.415     0.412     0.410      1807
weighted avg      0.431     0.414     0.419      1807

